<a href="https://colab.research.google.com/github/novello56prod/Produzione-25/blob/main/tempi_standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import urllib.parse
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

fasi = ["Taglio legno","Taglio Pannello","Assemblaggio","Imballaggio","Taglio"]

for  fase in fasi:
  sheet_names = [("Cassa in Pannello",1),("Cassa in Abete",2),("Basamento",3),("Pieghevole",4),("Gabbia",5)]
  sheet_id = "1URLjXqU3FQnccU3giVwxSPbigsJjLIIVXFPo2LfVlJY" # The sheet’s ID is the string located between d/ and /edit
  df_righe_commesse = pd.DataFrame() # Initialize an empty DataFrame
  for sheet_name in sheet_names:
    encoded_sheet_name = urllib.parse.quote(sheet_name[0])
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={encoded_sheet_name}"
    df_righe_commesse_tipo = pd.read_csv(url, header=0) # Explicitly set header to 0
    df_righe_commesse_tipo['Tipo imballo'] = sheet_name[1]
    df_righe_commesse = pd.concat([df_righe_commesse, df_righe_commesse_tipo], ignore_index=True)
#print(df_righe_commesse.columns)
  df_righe_commesse.rename(columns = {'        Tempi di lavorazione Taglio legno':'Taglio legno','Taglio Pannello ':'Taglio Pannello'}, inplace = True)
#print(df_righe_commesse.columns)
  df_righe_commesse = df_righe_commesse[['Tipo imballo','Superficie esterna', '% sup.pianta/sup.esterna', 'Tara','Pezzi da produrre','Taglio legno','Taglio Pannello', 'Assemblaggio', 'Imballaggio']]

  columns_to_convert = ['Tipo imballo','Superficie esterna', 'Tara', 'Pezzi da produrre','Taglio legno','Taglio Pannello', 'Assemblaggio', 'Imballaggio']
  for col in columns_to_convert:
    df_righe_commesse[col] = pd.to_numeric(df_righe_commesse[col], errors='coerce')
  df_righe_commesse.dropna(inplace=True)
  df_righe_commesse.reset_index(drop=True, inplace=True)

  lst=[]
  for index in df_righe_commesse.index:
    element = df_righe_commesse.loc[index,'Taglio legno']+df_righe_commesse.loc[index,'Taglio Pannello']
    lst.append(element)
  df_righe_commesse['Taglio'] = lst


  # create a Boolean mask for the rows to remove
  mask = df_righe_commesse['Tara'] == 0
  # select all rows except the ones that contain '0'
  df_righe_commesse = df_righe_commesse[~mask]

  mask = df_righe_commesse['Tara'] > 2000
  df_righe_commesse = df_righe_commesse[~mask]

  if fase == "Imballaggio":
      mask = df_righe_commesse['Imballaggio'] == 0
      df_righe_commesse = df_righe_commesse[~mask]

  if fase == 'Taglio Pannello': # Corrected the condition here
      mask = df_righe_commesse['Taglio Pannello'] == 0
      df_righe_commesse = df_righe_commesse[~mask]

  if fase == 'Taglio Pannello':
      mask = df_righe_commesse['Tipo imballo'] == 2
      df_righe_commesse = df_righe_commesse[~mask]
      mask = df_righe_commesse['Tipo imballo'] == 4
      df_righe_commesse = df_righe_commesse[~mask]

  #if fase == 'Taglio legno':
  #    mask = df_righe_commesse['Tipo imballo'] == 1
  #    df_righe_commesse = df_righe_commesse[~mask]

  mask = df_righe_commesse['% sup.pianta/sup.esterna'] == 0
  df_righe_commesse = df_righe_commesse[~mask]

  mask = df_righe_commesse['Pezzi da produrre'] == 0
  df_righe_commesse = df_righe_commesse[~mask]

  df_righe_commesse.reset_index(drop=True, inplace=True)

  def setup_time(pezzi,superficie):
      time = 0
      if superficie < 10:
        time = 5
      else : time = 20
      if pezzi < 5:
        time = time + (pezzi - 1) * 7
      elif pezzi >= 5 and pezzi < 15:
        time = time + (pezzi - 5) * 2
      elif pezzi >= 15 and pezzi < 51:
        time = time + 40
      elif pezzi > 50:
        time = time + (pezzi/50) * 20
      return time

  for i in range(len(df_righe_commesse)):
      df_righe_commesse.loc[i,'Setup'] = setup_time(df_righe_commesse.loc[i,'Pezzi da produrre'],df_righe_commesse.loc[i,'Superficie esterna'])
  df_righe_commesse['Setup'] = pd.to_numeric(df_righe_commesse['Setup'], errors='coerce')
  #df_righe_commesse.info()
  #df_righe_commesse.describe()

  #from google.colab import data_table

  #data_table.DataTable(
  #    df_righe_commesse, include_index=False, num_rows_per_page=30
 #)
  X = df_righe_commesse[['Superficie esterna', '% sup.pianta/sup.esterna', 'Tara', 'Tipo imballo']]
  y = df_righe_commesse[fase]

  for i in X.index:
    X.loc[i,'Superficie esterna'] = X.loc[i,'Superficie esterna']/179
    X.loc[i,'Tara'] = X.loc[i,'Tara']/2000
    X.loc[i,'Tipo imballo'] = X.loc[i,'Tipo imballo']/5
    X.loc[i,'Superficie esterna_2'] = X.loc[i,'Superficie esterna']**2
    X.loc[i,'% sup.pianta/sup.esterna_2'] = X.loc[i,'% sup.pianta/sup.esterna']**2
    X.loc[i,'Tipo imballo_2'] = X.loc[i,'Tipo imballo']**2
    X.loc[i,'Tara_2'] = X.loc[i,'Tara']**2


  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
#mse = mean_squared_error(y_test, y_pred)
#print("Mean Squared Error:", mse)
  coef = []
  coef = model.coef_
  intercept = model.intercept_
  dati = [{"Data": pd.Timestamp.today(),
        "Fase" : fase,
        "R2": model.score(X_test, y_test),
        "MSE": mean_squared_error(y_test, y_pred),
        "intercept": model.intercept_,
        "# coef": len(coef),
        'Superficie esterna': coef[0]/179,
        '% sup.pianta/sup.esterna': coef[1],
        'Tara': coef[2]/2000,
        'Tipo imballo': coef[3]/5,
        'Superficie esterna_2': coef[4],
        '% sup.pianta/sup.esterna_2': coef[5],
        'Tipo imballo_2': coef[6],
        'Tara_2': coef[7]

     }]

# Make data frame of above data
  df = pd.DataFrame(dati)
  df.to_csv('/content/drive/MyDrive/Esperimenti.csv',
                  mode='a',index=False, header=True)


/tmp/ipython-input-2523625991.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[i,'Tipo imballo'] = X.loc[i,'Tipo imballo']/5
/tmp/ipython-input-2523625991.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[i,'Superficie esterna_2'] = X.loc[i,'Superficie esterna']**2
/tmp/ipython-input-2523625991.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

      Superficie esterna  % sup.pianta/sup.esterna    Tara  Tipo imballo  \
0               0.017263                      0.23  0.0175           0.2   
1               0.021117                      0.22  0.0220           0.2   
2               0.110559                      0.22  0.1070           0.2   
3               0.036034                      0.13  0.0320           0.2   
4               0.107821                      0.20  0.0990           0.2   
...                  ...                       ...     ...           ...   
4450            0.148883                      0.18  0.1035           1.0   
4451            0.052067                      0.22  0.0370           1.0   
4452            0.060000                      0.32  0.0365           1.0   
4453            0.030279                      0.18  0.0185           1.0   
4454            0.067542                      0.28  0.0395           1.0   

      Superficie esterna_2  % sup.pianta/sup.esterna_2  Tipo imballo_2  \
0            

/tmp/ipython-input-2523625991.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[i,'Tipo imballo'] = X.loc[i,'Tipo imballo']/5
/tmp/ipython-input-2523625991.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[i,'Superficie esterna_2'] = X.loc[i,'Superficie esterna']**2
/tmp/ipython-input-2523625991.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

      Superficie esterna  % sup.pianta/sup.esterna    Tara  Tipo imballo  \
0               0.017263                      0.23  0.0175           0.2   
1               0.021117                      0.22  0.0220           0.2   
2               0.110559                      0.22  0.1070           0.2   
3               0.036034                      0.13  0.0320           0.2   
4               0.107821                      0.20  0.0990           0.2   
...                  ...                       ...     ...           ...   
1697            0.048939                      0.16  0.0240           1.0   
1698            0.049441                      0.16  0.0240           1.0   
1699            0.198547                      0.13  0.1380           1.0   
1700            0.237151                      0.14  0.1790           1.0   
1701            0.000782                      0.02  0.0795           1.0   

      Superficie esterna_2  % sup.pianta/sup.esterna_2  Tipo imballo_2  \
0            

/tmp/ipython-input-2523625991.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[i,'Tipo imballo'] = X.loc[i,'Tipo imballo']/5
/tmp/ipython-input-2523625991.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[i,'Superficie esterna_2'] = X.loc[i,'Superficie esterna']**2
/tmp/ipython-input-2523625991.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

      Superficie esterna  % sup.pianta/sup.esterna    Tara  Tipo imballo  \
0               0.017263                      0.23  0.0175           0.2   
1               0.021117                      0.22  0.0220           0.2   
2               0.110559                      0.22  0.1070           0.2   
3               0.036034                      0.13  0.0320           0.2   
4               0.107821                      0.20  0.0990           0.2   
...                  ...                       ...     ...           ...   
4450            0.148883                      0.18  0.1035           1.0   
4451            0.052067                      0.22  0.0370           1.0   
4452            0.060000                      0.32  0.0365           1.0   
4453            0.030279                      0.18  0.0185           1.0   
4454            0.067542                      0.28  0.0395           1.0   

      Superficie esterna_2  % sup.pianta/sup.esterna_2  Tipo imballo_2  \
0            

/tmp/ipython-input-2523625991.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[i,'Tipo imballo'] = X.loc[i,'Tipo imballo']/5
/tmp/ipython-input-2523625991.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[i,'Superficie esterna_2'] = X.loc[i,'Superficie esterna']**2
/tmp/ipython-input-2523625991.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

     Superficie esterna  % sup.pianta/sup.esterna    Tara  Tipo imballo  \
0              0.253352                      0.19  0.3000           0.2   
1              0.041397                      0.20  0.0480           0.2   
2              0.417430                      0.18  0.7400           0.2   
3              0.152067                      0.14  0.1670           0.2   
4              0.222011                      0.15  0.2380           0.2   
..                  ...                       ...     ...           ...   
549            0.157933                      0.18  0.1345           1.0   
550            0.295251                      0.20  0.3135           1.0   
551            0.100950                      0.18  0.1010           1.0   
552            0.117151                      0.14  0.1895           1.0   
553            0.192570                      0.18  0.1925           1.0   

     Superficie esterna_2  % sup.pianta/sup.esterna_2  Tipo imballo_2  \
0                0.064187 

/tmp/ipython-input-2523625991.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[i,'Tipo imballo'] = X.loc[i,'Tipo imballo']/5
/tmp/ipython-input-2523625991.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[i,'Superficie esterna_2'] = X.loc[i,'Superficie esterna']**2
/tmp/ipython-input-2523625991.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

      Superficie esterna  % sup.pianta/sup.esterna    Tara  Tipo imballo  \
0               0.017263                      0.23  0.0175           0.2   
1               0.021117                      0.22  0.0220           0.2   
2               0.110559                      0.22  0.1070           0.2   
3               0.036034                      0.13  0.0320           0.2   
4               0.107821                      0.20  0.0990           0.2   
...                  ...                       ...     ...           ...   
4450            0.148883                      0.18  0.1035           1.0   
4451            0.052067                      0.22  0.0370           1.0   
4452            0.060000                      0.32  0.0365           1.0   
4453            0.030279                      0.18  0.0185           1.0   
4454            0.067542                      0.28  0.0395           1.0   

      Superficie esterna_2  % sup.pianta/sup.esterna_2  Tipo imballo_2  \
0            